In [6]:
import numpy as np
import math
from interval import imath
from interval import fpu
from interval import interval
from functions import Functions

In [11]:
def left(interval):
    return fpu.max(interval)[0]


def right(interval):
    return fpu.max(interval)[1]


def mid(interval):
    return (fpu.max(interval)[0] + fpu.max(interval)[1]) / 2


def norm(p_1, p_2):
    r = 0
    for i in range(len(p_1)):
        r += (p_1[i] - p_2[i]) ** 2
    return math.sqrt(r)

Напишем алгоритм, который будет находить экстремум функции вдоль заданного направления.
Алгоритм будет основываться на методе "Золотого сечения" так как является наиболее эффективным по стравнению с основными методами одномерной оптимизации.

In [12]:
def GoldenRatio(func_index, index, a, b, p, e_d, e_f):  # f(function), i(index of direction),
    # a(left border), b(right border), p(current point), e_d(error of d), e_f(error of f)
    F = Functions[func_index * 2]
    phi = (1 + math.sqrt(5)) / 2  # constant of golden ratio
    x_1 = b - (b - a) / phi
    x_2 = a + (b - a) / phi
    p_1 = p.copy()  # current point
    p_2 = p.copy()  # current point
    p_1[index] = x_1
    p_2[index] = x_2
    f_1 = F(p_1)  # value in 1-st point
    f_2 = F(p_2)  # value in 2-nd point
    while (b - a > e_d) | (abs(f_1 - f_2) > e_f):  # termination criteria
        if f_1 <= f_2:
            b = x_2
            x_2 = x_1
            x_1 = b - (b - a) / phi

            p_1[index] = x_1
            p_2[index] = x_2

            f_2 = f_1
            f_1 = F(p_1)
        else:
            a = x_1
            x_1 = x_2
            x_2 = a + (b - a) / phi

            p_1[index] = x_1
            p_2[index] = x_2

            f_1 = f_2
            f_2 = F(p_2)

    best_point = []
    for i in range(len(p)):
        best_point.append((p_1[i] + p_2[i]) / 2)

    return best_point  # point of extremum with error e_d

Теперь реализуем алгоритм Moore-Skelboe

In [13]:
def MooreSkelboe(func_index, index, a, b, p, e_d, e_f):  # func_index(number of function in list of functions), index(index of direction),
    # a(left border), b(right border), p(current point), e_d(error of d), e_f(error of f)
    F = Functions[func_index * 2 + 1]
    interval_d = []
    for i in range(len(p)):
        interval_d.append(interval[p[i], p[i]])
    interval_d[index] = interval[a, b]

    interval_f = F(interval_d)
    set_of_intervals = [[interval_d, interval_f]]
    U = right(interval_f)
    w_f = right(interval_f) - left(interval_f)
    w_d = right(interval_d[index]) - left(interval_d[index])
    best_interval = set_of_intervals[0]
    while (w_d > e_d) | (w_f > e_f):
        set_of_intervals.pop(0)
        mid_p = mid(best_interval[0][index])
        interval_1 = best_interval[0].copy()
        interval_2 = best_interval[0].copy()
        interval_1[index] = interval[left(best_interval[0][index]), mid_p]
        interval_1_f = F(interval_1)
        interval_2[index] = interval[mid_p, right(best_interval[0][index])]
        interval_2_f = F(interval_2)
        U = min(U, right(interval_1_f))
        U = min(U, right(interval_2_f))

        for i in range(len(set_of_intervals)):
            if U < left(set_of_intervals[i][1]):
                set_of_intervals = set_of_intervals[:i]
                break

        set_of_intervals.append([interval_1, interval_1_f])
        set_of_intervals.append([interval_2, interval_2_f])

        set_of_intervals.sort(key=lambda item: left(item[1]))
        best_interval = set_of_intervals[0]
        w_f = right(best_interval[1]) - left(best_interval[1])
        w_d = right(best_interval[0][index]) - left(best_interval[0][index])

    best_point = []
    for i in range(len(p)):
        best_point.append(mid(best_interval[0][i]))

    return best_point

Функция вычисляющая евклидово расстояние между 2 точками (2,p норму)

Функция принимающая функцию, множество определения и начальную точку.
Она раелизует метод наискорейшего спуска, последовательно ищет минимум вдоль направлений параллельных координатным осям и останавливается когда улучшение точности становится малым.

In [14]:
def FastSearch(func_index, D, p, e_d, e_f, method):  # F(function), D(set), p(start point), e(error)
    while True:
        p_0 = p
        for index in range(len(p)):
            p = method(func_index, index, D[index][0], D[index][1], p, e_d, e_f)
        if (norm(p_0, p) < e_d) & (Functions[func_index * 2](p_0) - Functions[func_index * 2](p) < e_f):
            break

    best_point = p
    best_value = Functions[func_index * 2](p)

    return best_point, best_value

In [15]:
def print_result(func_num, D, p, e_d, e_f):
    p_1, v_1 = FastSearch(func_num, D, p, e_d, e_f, GoldenRatio)  # point of minimum
    p_2, v_2 = FastSearch(func_num, D, p, e_d, e_f, MooreSkelboe)  # point of minimum
    print("Результат тестирования:")
    print("Golden Ratio:", v_1)
    print("Moore-Skelboe:", v_2)


In [16]:
def print_result1(func_num, D, p, e_d, e_f):
    p_1, v_1 = FastSearch(func_num, D, p, e_d, e_f, GoldenRatio)  # point of minimum
    print("Результат тестирования:")
    print("Golden Ratio:", v_1)

In [17]:
def print_result2(func_num, D, p, e_d, e_f):
    p_2, v_2 = FastSearch(func_num, D, p, e_d, e_f, MooreSkelboe)  # point of minimum
    print("Результат тестирования:")
    print("Moore-Skelboe:", v_2)

In [8]:
# print_result(2, [[0, 4], [-2, 2]], [1, 1], 0.01, 0.01)  #считал минут 5

## Далее продемонстрируем работу методов на известных тестовых функциях глобальной оптимизации

In [59]:
% % time
# Функция Растригина
n = 40
l = -10
r = 10
D = [[l, r]] * n
p = [1] * n
print_result1(5, D, p, 0.001, 0.001)

Результат тестирования:
Golden Ratio: 39.79857020564907
CPU times: user 42.4 ms, sys: 2.43 ms, total: 44.8 ms
Wall time: 43.2 ms


In [104]:
% % time
# Функция Розенброка
n = 8
l = -10
r = 10
D = [[l, r]] * n
p = [2] * n
print_result1(7, D, p, 0.001, 0.001)

Результат тестирования:
Golden Ratio: 0.33244082803407593
CPU times: user 17.6 ms, sys: 1.84 ms, total: 19.5 ms
Wall time: 18.5 ms


In [11]:
# Функция Стыбинского-Танга
n = 4
l = -10
r = 10
D = [[l, r]] * n
p = [2] * n
print_result(12, D, p, 0.001, 0.001)

Результат тестирования:
Golden Ratio: -156.6646589598015
Moore-Skelboe: -156.66466281009914


In [120]:
% % time
# Функция Экли
n = 5
l = -10
r = 10
D = [[l, r]] * n
p = [2] * n
print_result2(15, D, p, 0.001, 0.001)

Результат тестирования:
Moore-Skelboe: 0.0012256630393632229
CPU times: user 527 ms, sys: 6.59 ms, total: 534 ms
Wall time: 553 ms


In [29]:
% % time
# Функция Гриванка
n = 5
l = -10
r = 10
D = [[l, r]] * n
p = [2] * n
print_result1(16, D, p, 0.001, 0.001)

Результат тестирования:
Golden Ratio: 0.009864698515380743
CPU times: user 2.26 ms, sys: 162 µs, total: 2.42 ms
Wall time: 2.41 ms


In [82]:
% % time
# Функция Растригина новгородская
n = 5
l = -10
r = 10
D = [[l, r]] * n
p = [2] * n
print_result2(17, D, p, 0.001, 0.001)

Результат тестирования:
Moore-Skelboe: 0.0007449817996270092
CPU times: user 149 ms, sys: 4.78 ms, total: 154 ms
Wall time: 151 ms


In [44]:
% % time
# Функция Швефеля
n = 2
l = -500
r = 500
D = [[l, r]] * n
p = [2] * n
print_result1(18, D, p, 0.001, 0.001)

Результат тестирования:
Golden Ratio: 236.8766946858181
CPU times: user 607 µs, sys: 113 µs, total: 720 µs
Wall time: 721 µs


### Реализуем теперь градиентные алгоритмы

In [32]:
def Gradient(D, p, f, step):
    gradient = np.array([None] * len(p))
    for i in range(len(p)):
        p_ = p.copy()
        p_[i] += step
        gradient[i] = (f(p_) - f(p)) / step
        if (abs(p[i] - D[i][0]) < step) & (p[i] < 0):
            p[i] = 0
        if (abs(p[i] - D[i][1]) < step) & (p[i] > 0):
            p[i] = 0
    return gradient

In [41]:
def FastSearchG(func_index, D, p, e_d, e_f, method):  # F(function), D(set), p(start point), e(error)
    F = Functions[func_index * 2]
    gradient = Gradient(D, p, F, e_d)
    while np.linalg.norm(gradient) > e_f:
        p_0 = p
        p = method(func_index, gradient, D, p, e_d, e_f)
        gradient = Gradient(D, p, F, e_d)
        if norm(p, p_0) < e_d:
            break

    best_point = p
    best_value = Functions[func_index * 2](p)

    return best_point, best_value

In [42]:
def Borders(p, gradient, D):
    max_t = math.inf
    min_t = -math.inf
    dir = gradient / np.linalg.norm(gradient)
    for i in range(len(p)):
        if dir[i] > 0:
            max_t = min((D[i][1] - p[i]) / dir[i], max_t)
            min_t = max((D[i][0] - p[i]) / dir[i], min_t)
        elif dir[i] < 0:
            min_t = max((D[i][1] - p[i]) / dir[i], min_t)
            max_t = min((D[i][0] - p[i]) / dir[i], max_t)

    borders = [p + dir * min_t, p + dir * max_t]
    return borders

In [43]:
def GoldenRatioG(func_index, gradient, D, p, e_d, e_f):  # f(function), i(index of direction),
    # a(left border), b(right border), p(current point), e_d(error of d), e_f(error of f)
    F = Functions[func_index * 2]
    phi = (1 + math.sqrt(5)) / 2  # constant of golden ratio
    a = Borders(p, gradient, D)[0]
    b = Borders(p, gradient, D)[1]
    x_1 = b - (b - a) / phi
    x_2 = a + (b - a) / phi
    p_1 = x_1
    p_2 = x_2
    f_1 = F(p_1)  # value in 1-st point
    f_2 = F(p_2)  # value in 2-nd point
    while (norm(b, a) > e_d) | (abs(f_1 - f_2) > e_f):  # termination criteria
        if f_1 <= f_2:
            b = x_2
            x_2 = x_1
            x_1 = b - (b - a) / phi

            p_1 = x_1
            p_2 = x_2

            f_2 = f_1
            f_1 = F(p_1)
        else:
            a = x_1
            x_1 = x_2
            x_2 = a + (b - a) / phi

            p_1 = x_1
            p_2 = x_2

            f_1 = f_2
            f_2 = F(p_2)

    best_point = []
    for i in range(len(p)):
        best_point.append((p_1[i] + p_2[i]) / 2)

    return best_point  # point of extremum with error e_d

In [44]:
print(FastSearchG(0, [[-10,10], [-10, 10]], [1, 1], 0.01, 0.01, GoldenRatioG))

([-0.00999749259584825, 0], 3.000049974929102)


In [45]:
print(FastSearchG(1, [[-10,10], [-10, 10]], [1, 1], 0.01, 0.01, GoldenRatioG))

([-0.0010696331036035857, -0.0010696331036035857], 0.004339463348134842)


In [46]:
print(FastSearchG(2, [[-10,10], [-10, 10]], [1, 1], 0.01, 0.01, GoldenRatioG))

([-0.09769963899210414, 3.184885152583509], 3.683754359610854)


In [47]:
print(FastSearchG(3, [[-10,10], [-10, 10]], [1, 1], 0.01, 0.01, GoldenRatioG))

([1.0005450029007914, 2.9988407577841043], 3.1500304218649655e-06)
